In [1]:
import pandas as pd
import torch
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv("Demo-Annotations-R1-2020-08-21.csv")

In [3]:
def process_text(s):
    s = s.replace("Claim1:\n\n", "")
    return s.split("\n\nClaim2:\n\n")

In [4]:
df['example'] = df['text'].apply(process_text)

In [5]:
def normalize_tags(s):
    s = s.replace("STRICT_", "")
    return s.lower()

In [6]:
df['label'] = df['tags'].apply(normalize_tags)

In [7]:
model = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')

Using cache found in /Users/cgpotts/.cache/torch/hub/pytorch_fairseq_master


In [8]:
_ = model.eval()

In [9]:
X = df['example'].apply(lambda x: model.encode(*x))

In [10]:
pred_indices = [model.predict('mnli', ex).argmax() for ex in X]

In [11]:
to_str = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}

In [12]:
preds = [to_str[c.item()] for c in pred_indices]

In [13]:
gold = df['label'].values

In [14]:
print(classification_report(gold, preds))

               precision    recall  f1-score   support

contradiction       0.75      0.15      0.25        20
   entailment       0.50      0.04      0.08        23
      neutral       0.34      0.95      0.51        21

     accuracy                           0.38        64
    macro avg       0.53      0.38      0.28        64
 weighted avg       0.53      0.38      0.27        64

